In [496]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [498]:
df_train = pd.read_csv('Wyoming Census data 2016.csv')
df_train_target=pd.read_csv('Wyoming total votes 2016.csv')
df_test = pd.read_csv('Wyoming Census data 2020.csv')
df_test_target=pd.read_csv('Wyoming total votes 2020.csv')
#df_test.drop(0,axis=0,inplace=True)
print(df_train_target)
# df_train=df_train
# df_test=df_test
print(df_train)

        Party   Votes
0  Republican  174419
1    Democrat   55973
2       Other   25457
                           Age groups  Under 5 years  5 to 9 years  \
0  Percentage of total population (%)            6.5           6.9   

   10 to 14 years  15 to 19 years  20 to 24 years  25 to 29 years  \
0             6.6             6.3             7.2               7   

   30 to 34 years  35 to 39 years  40 to 44 years  45 to 49 years  \
0               7             6.2               6             5.8   

   50 to 54 years  55 to 59 years  60 to 64 years  65 to 69 years  \
0             6.8             7.1             6.7             4.8   

   70 to 74 years  75 to 79 years  80 to 84 years  85 years and over  
0             3.4             2.3             1.7                1.7  


In [500]:
#useful functions
def swap_rows(df, row1, row2):
    df.iloc[row1], df.iloc[row2] =  df.iloc[row2].copy(), df.iloc[row1].copy()
    return df

In [502]:
df_train.index=['Percentage'] #nice


In [504]:
#Drop 1st column from training and test sets
df_train=df_train.drop(df_train.columns[0],axis=1)
df_train

,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
Percentage,6.5,6.9,6.6,6.3,7.2,7,7,6.2,6,5.8,6.8,7.1,6.7,4.8,3.4,2.3,1.7,1.7


In [506]:
# df_train = df_train.iloc[1:].reset_index(drop=True)
# df_test = df_test.iloc[1:].reset_index(drop=True)
#print(df_train,'\n',df_test)

In [508]:
votes_train=df_train_target['Votes'] #target
votes_test=df_train_target['Votes']
votes_train=np.array(votes_train) #make into numpy array
votes_test=np.array(votes_test)
type(votes_test)

numpy.ndarray

In [510]:
df_train

,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
Percentage,6.5,6.9,6.6,6.3,7.2,7,7,6.2,6,5.8,6.8,7.1,6.7,4.8,3.4,2.3,1.7,1.7


In [512]:
#Make sure every number is a float and not a string
# df_train['Percentage']=df_train['Percentage'].astype(float)
# #df_test['Percentage of total population (%)']=df_test['Percentage of total population (%)'].astype(float)


In [514]:
midpoints=[2.5,7,12,17,22,27,32,37,42,47,52,57,62,67,72,77,82,90]
df_train.loc[len(df_train)]=midpoints

In [516]:
df_train

,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
Percentage,6.5,6.9,6.6,6.3,7.2,7.0,7.0,6.2,6.0,5.8,6.8,7.1,6.7,4.8,3.4,2.3,1.7,1.7
1,2.5,7.0,12.0,17.0,22.0,27.0,32.0,37.0,42.0,47.0,52.0,57.0,62.0,67.0,72.0,77.0,82.0,90.0


In [526]:
df_train=df_train.loc[[1,'Percentage']] #nice

In [528]:
df_train

,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
1,2.5,7.0,12.0,17.0,22.0,27.0,32.0,37.0,42.0,47.0,52.0,57.0,62.0,67.0,72.0,77.0,82.0,90.0
Percentage,6.5,6.9,6.6,6.3,7.2,7.0,7.0,6.2,6.0,5.8,6.8,7.1,6.7,4.8,3.4,2.3,1.7,1.7


In [530]:
df_train.index=['Midpoints','Percentage']
df_train

,Under 5 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,50 to 54 years,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 years and over
Midpoints,2.5,7.0,12.0,17.0,22.0,27.0,32.0,37.0,42.0,47.0,52.0,57.0,62.0,67.0,72.0,77.0,82.0,90.0
Percentage,6.5,6.9,6.6,6.3,7.2,7.0,7.0,6.2,6.0,5.8,6.8,7.1,6.7,4.8,3.4,2.3,1.7,1.7


In [532]:
#####################COOL SO FAR###########################

In [1]:
#Test line for commit

In [534]:
#Find mean age
ages=df_train['Age groups']
# ages=np.array(ages)
# type(ages)

df_train['Midpoint']=midpoints
df_test['Midpoint']=midpoints
df_train=df_train.reindex(columns=['Age groups', 'Midpoint','Percentage of total population (%)'])
dfdf_test.reindex(columns=['Age groups', 'Midpoint','Percentage of total population (%)'])

KeyError: 'Age groups'

In [52]:
df_train.drop(['Age groups'],axis=1,inplace=True)

In [54]:
average_age_train=(df_train['Midpoint'] * df_train['Percentage of total population (%)']).sum()/df_train['Percentage of total population (%)'].sum()
average_age_test=(df_test['Midpoint'] * df_test['Percentage of total population (%)']).sum()/df_test['Percentage of total population (%)'].sum()

print(average_age_train,average_age_test)

37.8585 38.82919180902159


In [56]:
##OPTIONAL, IF WE WANT TO KEEP ONLY 18+ ## (DOESNT CHANGE RESULTS)
# df_train.drop([0,1,2],axis=0,inplace=True)
# print(df_train)
# df_test.drop([0,1,2],axis=0,inplace=True)
# print(df_test)

In [58]:
print(df_train_target)

        Party   Votes
0  Republican  174419
1    Democrat   55973
2       Other   25457


In [90]:
## Linear Regression ##
#1. Load Data
X=df_train.values[:,:] #Matrix of [midpoint age,percentage]
y=df_train_target.values[:,1] #Array of votes of 3 choices: 1. Republican, 2. Democrat, 3. Other
print(X,y)

[[ 2.5  6.5]
 [ 7.   6.9]
 [12.   6.6]
 [17.   6.3]
 [22.   7.2]
 [27.   7. ]
 [32.   7. ]
 [37.   6.2]
 [42.   6. ]
 [47.   5.8]
 [52.   6.8]
 [57.   7.1]
 [62.   6.7]
 [67.   4.8]
 [72.   3.4]
 [77.   2.3]
 [82.   1.7]
 [90.   1.7]] [174419 55973 25457]


In [92]:
#2. Model
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [18, 3]